<a href="https://colab.research.google.com/github/RiazMahmood6/E-book/blob/main/E_book_Generate_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import openai
import time
import os
import requests
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY
from ebooklib import epub

class BookGenerator:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        # Enhanced styles
        self.styles = getSampleStyleSheet()
        # Add custom styles for a more beautiful PDF
        self.styles.add(ParagraphStyle(
            name='ChapterTitle',
            parent=self.styles['Heading1'],
            fontSize=20,
            spaceAfter=30,
            spaceBefore=50,
            alignment=TA_CENTER
        ))
        self.styles.add(ParagraphStyle(
            name='BookTitle',
            parent=self.styles['Title'],
            fontSize=28,
            spaceAfter=30,
            alignment=TA_CENTER
        ))
        self.styles.add(ParagraphStyle(
            name='BodyJustified',
            parent=self.styles['BodyText'],
            fontSize=12,
            leading=14,
            alignment=TA_JUSTIFY,
            firstLineIndent=20
        ))
        self.styles.add(ParagraphStyle(
            name='TOCHeading',
            parent=self.styles['Heading1'],
            fontSize=18,
            spaceAfter=20,
            spaceBefore=30,
            alignment=TA_CENTER
        ))
    # 1. Update the generate_chapter method to ensure it creates chapter titles:
    def generate_chapter(self, title, description, chapter_num, target_words, language="en", prev_content=""):
        prompt = f"""
        You are an expert author writing a book titled '{title}'.
        The book description is: '{description}'.

        Write Chapter {chapter_num} (~{target_words} words) in {language} (ISO code: {language}).
        Your chapter MUST have a creative title that reflects its content.

        Format your response EXACTLY like this:
        # Chapter {chapter_num}: [Your creative chapter title here]

        [chapter content starts here]

        Ensure logical story progression. Previous content summary: {prev_content[:500]}
        """
        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a creative writer. Always create meaningful chapter titles for your stories."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=int(target_words / 0.75),
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"# Chapter {chapter_num}: Error Occurred\n[Error: {str(e)}]"

    # 2. Update the generate_cover_image method to create appropriate covers:
    def generate_cover_image(self, title, description):
        prompt = f"""
        Create a minimalist book cover image for '{title}'.
        Description: {description}

        IMPORTANT:
        - DO NOT include any author names, publisher information, or fake reviews
        - DO NOT make it look like an existing published book
        - DO NOT include any text other than possibly the title
        - Create a simple artistic design that represents the book's theme
        - Use abstract imagery, simple symbols, or minimalist art style
        - The cover should be clean, modern, and professional
        """

        max_retries = 3
        retries = 0

        while retries < max_retries:
            try:
                response = self.client.images.generate(
                    model="dall-e-3",
                    prompt=prompt,
                    n=1,
                    size="1024x1024"
                )
                image_url = response.data[0].url
                image_path = f"{title.replace(' ', '_')}_cover.jpg"
                with open(image_path, 'wb') as f:
                    f.write(requests.get(image_url).content)
                return image_path
            except Exception as e:
                print(f"Cover image error: {str(e)}")
                retries += 1
                time.sleep(1)

        print(f"Failed to generate cover image after {max_retries} retries.")
        return None

    def generate_book(self, title, description, target_pages=50, language="en"):
        words_per_page = 250
        total_words = target_pages * words_per_page
        words_per_chapter = 2000
        num_chapters = max(1, total_words // words_per_chapter)

        print(f"Generating '{title}' ({language}, ~{target_pages} pages, ~{num_chapters} chapters)")

        book_content = []
        prev_content = ""
        for chapter in range(1, num_chapters + 1):
            print(f"Chapter {chapter}...")
            chapter_content = self.generate_chapter(title, description, chapter, words_per_chapter, language, prev_content)
            book_content.append(chapter_content)
            prev_content = chapter_content
            time.sleep(2)

        cover_image_path = self.generate_cover_image(title, description)
        return book_content, cover_image_path

    def save_book_as_pdf(self, title, content_list, cover_image_path):
        filename = f"{title.replace(' ', '_')}.pdf"
        doc = SimpleDocTemplate(
            filename,
            pagesize=letter,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        story = []

        # Title Page
        if cover_image_path:
            title_paragraph = Paragraph(f"{title}", self.styles['BookTitle'])
            story.append(title_paragraph)
            story.append(Spacer(1, 0.2 * inch))

            # Calculate max dimensions with margins
            max_width = letter[0] - 2.5 * inch
            max_height = letter[1] - 3.5 * inch
            img = Image(cover_image_path, width=max_width, height=max_height)
            img.hAlign = 'CENTER'
            story.append(img)
        story.append(PageBreak())

        # Table of Contents with better styling
        story.append(Paragraph("Table of Contents", self.styles['TOCHeading']))
        story.append(Spacer(1, 0.25 * inch))

        toc_data = [["Chapter", "Page"]]
        page_num = 2
        for i, chapter in enumerate(content_list, 1):
            toc_data.append([f"Chapter {i}", str(page_num)])
            page_num += len(chapter.split('\n')) // 30 + 1  # Rough estimation

        toc_table = Table(toc_data, colWidths=[4*inch, 1*inch])
        toc_table.setStyle(TableStyle([
            ('GRID', (0, 0), (-1, -1), 0.5, colors.white),
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.darkblue),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('LINEBELOW', (0, 0), (-1, -1), 1, colors.lightgrey),
        ]))
        story.append(toc_table)
        story.append(PageBreak())

        # Chapters with improved styling
        for chapter in content_list:
            lines = chapter.split('\n')
            chapter_title = ""
            content_lines = []

            for line in lines:
                if line.startswith('# Chapter'):
                    chapter_title = line.replace('# ', '')
                else:
                    content_lines.append(line)

            story.append(Paragraph(chapter_title, self.styles['ChapterTitle']))
            story.append(Spacer(1, 0.25 * inch))

            content_text = ' '.join(content_lines)
            paragraphs = content_text.split('\n\n')

            for para in paragraphs:
                if para.strip():
                    story.append(Paragraph(para, self.styles['BodyJustified']))
                    story.append(Spacer(1, 0.15 * inch))

            story.append(PageBreak())

        doc.build(story)
        print(f"PDF created: {filename}")
        return filename

    def save_book_as_epub(self, title, content_list, cover_image_path):
        filename = f"{title.replace(' ', '_')}.epub"
        book = epub.EpubBook()
        book.set_identifier(f'id_{title.replace(" ", "_")}')
        book.set_title(title)
        book.set_language('en')
        book.add_author('AI Author')

        if cover_image_path:
            with open(cover_image_path, 'rb') as f:
                book.set_cover("cover.jpg", f.read())

            title_page = epub.EpubHtml(title="Cover", file_name="cover.xhtml", lang='en')
            title_page.content = f'''
            <html>
                <head>
                    <style>
                        body {{ text-align: center; padding: 20px; }}
                        h1 {{ font-size: 24pt; color: #333; margin-bottom: 30px; }}
                        img {{ max-width: 90%; height: auto; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }}
                    </style>
                </head>
                <body>
                    <h1>{title}</h1>
                    <img src="cover.jpg" alt="Cover Image" />
                </body>
            </html>'''
            book.add_item(title_page)

            # Table of Contents with better styling and chapter titles
            story.append(Paragraph("Table of Contents", self.styles['TOCHeading']))
            story.append(Spacer(1, 0.25 * inch))

            # Improved table of contents code to extract chapter titles:
            toc_data = [["Chapter", "Page"]]
            page_num = 2
            for i, chapter in enumerate(content_list, 1):
                # Extract the chapter title from the content
                lines = chapter.split('\n')
                chapter_title = f"Chapter {i}"

                # Find the line with the chapter heading
                for line in lines:
                    if line.startswith('# Chapter'):
                        # If it has a format like "# Chapter X: Title"
                        if ':' in line:
                            chapter_title = line.replace('# ', '')
                        break

                toc_data.append([chapter_title, str(page_num)])
                page_num += len(chapter.split('\n')) // 30 + 1  # Rough estimation

            toc_table = Table(toc_data, colWidths=[4*inch, 1*inch])
            toc_table.setStyle(TableStyle([
                ('GRID', (0, 0), (-1, -1), 0.5, colors.white),
                ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.darkblue),
                ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left-align chapter names
                ('ALIGN', (1, 0), (1, -1), 'CENTER'), # Center page numbers
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                ('LINEBELOW', (0, 0), (-1, -1), 1, colors.lightgrey),
            ]))


        # Define Table of Contents
        book.toc = tuple(chapters)
        book.spine = ['nav'] + chapters
        book.add_item(epub.EpubNcx())
        book.add_item(epub.EpubNav())

        # Write the EPUB file
        epub.write_epub(filename, book)
        print(f"EPUB created: {filename}")
        return filename


# Example usage for command-line/GitHub version (no widgets)
def generate_book_from_command_line(api_key, title, description, target_pages=50, language="en", output_format="PDF"):
    book_gen = BookGenerator(api_key)

    print(f"Generating book: {title}")
    print(f"Description: {description}")
    print(f"Pages: {target_pages}")
    print(f"Language: {language}")
    print(f"Format: {output_format}")

    book_content, cover_image_path = book_gen.generate_book(title, description, target_pages, language)

    if output_format in ["PDF", "Both"]:
        pdf_file = book_gen.save_book_as_pdf(title, book_content, cover_image_path)
        print(f"PDF file generated: {pdf_file}")

    if output_format in ["EPUB", "Both"]:
        epub_file = book_gen.save_book_as_epub(title, book_content, cover_image_path)
        print(f"EPUB file generated: {epub_file}")


# Replace the command-line argument section with this simpler approach for Colab
# Delete everything from "if __name__ == "__main__":" to the end and replace with:

# Function to generate book without widgets in Colab
def generate_book_simple(api_key, title="The Quantum Chronicles",
                         description="A sci-fi adventure across dimensions.",
                         target_pages=50, language="en", output_format="PDF"):
    book_gen = BookGenerator(api_key)

    print(f"Generating book: {title}")
    print(f"Description: {description}")
    print(f"Pages: {target_pages}")
    print(f"Language: {language}")
    print(f"Format: {output_format}")

    book_content, cover_image_path = book_gen.generate_book(title, description, target_pages, language)

    from google.colab import files
    if output_format in ["PDF", "Both"]:
        pdf_file = book_gen.save_book_as_pdf(title, book_content, cover_image_path)
        files.download(pdf_file)

    if output_format in ["EPUB", "Both"]:
        epub_file = book_gen.save_book_as_epub(title, book_content, cover_image_path)
        files.download(epub_file)




# **Give input to generate book**

In [5]:
# Set your parameters here and run the cell
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# Run this function to generate your book
generate_book_simple(
    api_key=openai_api_key,
    title="3 ways of making money with AI",
    description="Discover innovative strategies to profit from artificial intelligence",
    target_pages=50, #50 = 6 chapters
    language="en",
    output_format="PDF"  # Options: "PDF", "EPUB", or "Both"
)

Generating book: 3 ways of making money with AI
Description: Discover innovative strategies to profit from artificial intelligence
Pages: 50
Language: en
Format: PDF
Generating '3 ways of making money with AI' (en, ~50 pages, ~6 chapters)
Chapter 1...
Chapter 2...
Chapter 3...
Chapter 4...
Chapter 5...
Chapter 6...
PDF created: 3_ways_of_making_money_with_AI.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>